In [ ]:
import os
import pandas as pd
from tqdm.std import tqdm

In [ ]:
data_path = "document/top1000.dev"

In [ ]:
data = pd.read_csv(data_path,sep="\t", names=["qid","pid","query","passage"])

In [ ]:
data.head(10)

In [ ]:
len(data)

In [ ]:
data_groups = data.groupby("qid").size()

In [ ]:
data_groups

In [ ]:
# Check if each qid corresponds to 1000 instances

invalid_qids = data_groups[data_groups > 100]
print("The following qids do not meet the criteria：")
print(invalid_qids)

In [ ]:
large_qid = invalid_qids.index

In [ ]:
large_qid

## qrels

In [ ]:
dev_qrels = pd.read_csv("document/qrels.dev.tsv",sep="\t", names=["qid", "iteration", "pid", "relevance_score"])

In [ ]:
dev_qrels

In [ ]:
dev_qrels_group = dev_qrels.groupby("qid")

In [ ]:
dev_qrels_group.size()

## filter all positive instance

In [ ]:
filter_positive_df = pd.merge(left=data,right=dev_qrels,on=["pid","qid"],how='left')

In [ ]:
filter_positive_df

In [ ]:
new_data = filter_positive_df[["qid","pid","query","passage","relevance_score"]]

In [ ]:
new_data.fillna(0,inplace=True)

In [ ]:
new_data

In [ ]:
new_data = new_data[new_data.qid.isin(large_qid)]

In [ ]:
new_data.reset_index(drop=True,inplace=True)

In [ ]:
new_data

In [ ]:
filter_results = []

for qid, group in tqdm(new_data.groupby("qid")):
    
    ones = group[group["relevance_score"] == 1.0]
    zeros = group[group["relevance_score"] == 0.0]

    # Randomly sample the required number of rows from zeros
    required_zeros = 100 - len(ones)
    sampled_zeros = zeros.sample(n=required_zeros, random_state=42, replace=False)

    # Combine the results
    new_group = pd.concat([ones, sampled_zeros])
    filter_results.append(new_group)

In [ ]:
final_df = pd.concat(filter_results).reset_index(drop=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv("document/filtered_passage_reranking_data.tsv", sep="\t", index=False)